# TerrainPatterns
Let's patternd the world

In [ ]:
#| default_exp terrainpatterns

### To Do's

1. Create a new resource group for triangles
1. Fix contour so it only does the high side

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# unique
from treelib import Tree
import heapq

In [ ]:
#| export

from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, StyleDemo, Generatable
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexRegion, HexWrapper, HexPosition, PrimitiveDemo, hexBackground

In [ ]:
#| export
from HexMagic.terrain import Terrain, TerraDemo

### Plot Thickens

Based on the code context, here's a summary of how overlay plotting works in this terrain visualization system:

**Overlay Plotting Summary**

The overlay system allows you to visualize additional data layers (like rainfall, temperature) on top of the base terrain elevation map using SVG patterns.

**Key Components:**

1. **Pattern Generation** (`ballPatterns`): Creates a set of SVG pattern definitions, typically representing different intensity levels. Each pattern is a scaled circle whose radius increases with the level index.

2. **Data Binning** (`rainfall_selector_np`): Converts continuous data values into discrete pattern indices using `np.digitize`. For example, rainfall amounts are binned into categories (0-5, 5-12, 12-24, 24-48, 48+ inches).

3. **Overlay Application** (`makeOverlay`): 
   - Adds pattern definitions to the SVG builder
   - Iterates through each hex cell
   - Applies the appropriate pattern as a fill based on the data index
   - Creates polygon elements with `url(#pattern_id)` fills

**Workflow:**
```python
# 1. Prepare your data field
rainfall_data = np.random.uniform(-4, 60, len(terrain.elevations))
terrain.fields['rainfall'] = rainfall_data

# 2. Bin the data into pattern indices
pattern_indices = rainfall_selector_np(rainfall_data)

# 3. Generate patterns and apply overlay
patterns = ballPatterns(5)  # 5 intensity levels
terrain.makeOverlay(pattern_indices, patterns)
```

The result is a dual-layer visualization: colored hexes showing elevation beneath patterned overlays showing your additional data field.

In [ ]:
#| export
class TerrainPatterns:

    def __init__(self,terrain):
        self.terrain = terrain



    def ballScale(self, levels=6,fills=["#007fff"],prefix="ball")->[SVGBuilder]:
        ret = []
        for i in range(levels):
            colorIndex = min(i,len(fills)-1)
            fill = fills[colorIndex]
            
        
            ballDim = 60
            style = StyleCSS(f"{prefix}_{i}",fill=fill)
            aBuilder = SVGBuilder()
            radius = 20
            
            body = f"""
            <g>
                <ellipse cx="{ballDim/2}" cy="{ballDim/2}" rx="{radius}" ry="{radius}" style="fill:{fill};"/>
            </g>
        """
            aBuilder.blockTag = "pattern"
            aBuilder.width = ballDim
            aBuilder.height = ballDim
            scale = 0.1 + i * 0.8 / levels
            aBuilder.attributes = {
                'id': f"fill_{i}",
                'patternUnits': 'userSpaceOnUse',
                'patternTransform': f'scale({scale})'
            }
            aBuilder.updateLayers([body])

            ret.append(aBuilder)
        return ret

   





In [ ]:
#| export
@patch
def ballDensity(self:TerrainPatterns, levels=6, fills=["#007fff"], prefix="ball") -> [SVGBuilder]:
    """Create density patterns using circles of increasing size."""
    ret = []
    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
    
        ballDim = 60
        aBuilder = SVGBuilder()
        radius = min(i*56/levels + 2, ballDim/2-1)
        
        body = f"""
        <g>
            <circle cx="{ballDim/2}" cy="{ballDim/2}" r="{radius}" style="fill:{fill};"/>
        </g>
        """
        aBuilder.blockTag = "pattern"
        aBuilder.width = ballDim
        aBuilder.height = ballDim
        scale = 0.1
        aBuilder.attributes = {
            'id': f"{prefix}_{i}",  # USE THE PREFIX!
            'patternUnits': 'userSpaceOnUse',
            'patternTransform': f'scale({scale})'
        }
        aBuilder.updateLayers([body])

        ret.append(aBuilder)
    return ret

In [ ]:
#| export
@patch
def ballSpectrum(self:TerrainPatterns, levels=5, fills=["#007fff"], prefix="ball") -> [SVGBuilder]:
    """Create density patterns using circles of increasing size."""
    ret = []
    

    for i in range(levels):
        colorIndex = min(i, len(fills)-1)
        fill = fills[colorIndex]
    
        ballDim = 60
        biggest = ballDim/2-1 
        smallest = 5
        delta = (biggest - smallest)/levels * 2
        

        aBuilder = SVGBuilder()
        if i < levels/2:
            radius = biggest - delta * i
        elif i > levels/2:
            k = levels - i
            radius = biggest - delta * k
        else:
            radius = smallest



        
        body = f"""
        <g>
            <circle cx="{ballDim/2}" cy="{ballDim/2}" r="{radius}" style="fill:{fill};"/>
        </g>
        """
        aBuilder.blockTag = "pattern"
        aBuilder.width = ballDim
        aBuilder.height = ballDim
        scale = 0.1
        aBuilder.attributes = {
            'id': f"{prefix}_{i}",  # USE THE PREFIX!
            'patternUnits': 'userSpaceOnUse',
            'patternTransform': f'scale({scale})'
        }
        aBuilder.updateLayers([body])

        ret.append(aBuilder)
    return ret

In [ ]:
@patch
def dotDemo(self:TerraDemo):
    sampleMap = TerraDemo().tiny()
    terrainPatterns = TerrainPatterns(sampleMap)
    for x in terrainPatterns.ballDensity():
        print(x.xml())

    

In [ ]:
TerraDemo().dotDemo()

<?xml version='1.0' encoding='utf-8'?>
<pattern id="ball_0" patternUnits="userSpaceOnUse" patternTransform="scale(0.1)" width="60" height="60" viewBox="0 0 60 60" xmlns="http://www.w3.org/2000/svg">
<title> Untitled </title>


        <g>
            <circle cx="30.0" cy="30.0" r="2.0" style="fill:#007fff;"/>
        </g>
        
</pattern>
<?xml version='1.0' encoding='utf-8'?>
<pattern id="ball_1" patternUnits="userSpaceOnUse" patternTransform="scale(0.1)" width="60" height="60" viewBox="0 0 60 60" xmlns="http://www.w3.org/2000/svg">
<title> Untitled </title>


        <g>
            <circle cx="30.0" cy="30.0" r="11.333333333333334" style="fill:#007fff;"/>
        </g>
        
</pattern>
<?xml version='1.0' encoding='utf-8'?>
<pattern id="ball_2" patternUnits="userSpaceOnUse" patternTransform="scale(0.1)" width="60" height="60" viewBox="0 0 60 60" xmlns="http://www.w3.org/2000/svg">
<title> Untitled </title>


        <g>
            <circle cx="30.0" cy="30.0" r="20.666666666666

In [ ]:
#| export
@patch
def makeOverlay(self:Terrain,data,patterns:[SVGBuilder])->str:
    testBody = ""
    grid = self.hexGrid

    addSet = set()

    for i, patIndex in enumerate(data):
        if patIndex >= 0 and patIndex < len(patterns):
            patName = patterns[patIndex].attributes['id']
            addSet.add(patIndex)
    
            fill = f"url(#{patName})"

            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"

            testBody += "\t" +  ret + "\n"

    aList = list(addSet)
    aList.sort()
    for patIndex in aList:
        grid.builder.add_definition(patterns[patIndex])

    return testBody
   
        

In [ ]:
#| export
# Create a terrain
@patch
def circusDemo(self:TerraDemo):
    sampleMap = TerraDemo().tiny()

    def rainfall_selector_np(values: np.ndarray) -> np.ndarray:
        # bin edges: <5 returns 0, 5-12 returns 1, 12-24 returns 2, etc.
        bins = [0.1, 5, 12, 24, 48]
        return np.digitize(values, bins) - 1 

    test_data = np.array([-1, 2, 8, 15, 30, 60, 0, 12, 48])

    indices = rainfall_selector_np(test_data)
    indices

    # Generate random rainfall data
    rainfall_data = np.random.uniform(-4, 60, len(sampleMap.elevations))
    sampleMap.fields['rainfall'] = rainfall_data

    # Get pattern indices
    pattern_indices = rainfall_selector_np(rainfall_data)

    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballScale(len(pattern_indices),fills=["#007fff","#d4ff00ff","#ee00ffff","#ff0099ff","#00ff1eff"])  # 5 levels
    patot = sampleMap.makeOverlay(pattern_indices, patterns)
    sampleMap.hexGrid.builder.adjust("rainfall",patot)
    #sampleMap.colorMap()
    #sampleMap.hexGrid.update()

    # View it
    #print(sampleMap.hexGrid.builder.xml())
    return sampleMap.hexGrid.builder.show()


In [ ]:
TerraDemo().circusDemo()

In [ ]:
#| export
@patch
def fillPattern(self:HexRegion,pattern:SVGBuilder,smooth=False):
    """Fill a region with a style.

     fill = f"url(#{patName})"

            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
    
    
    """
    
    fill_layer = ""
    patName = pattern.attributes['id']
    fill = f"url(#{patName})"
    #fill = "#d4ff00eb"

    paths, gaps = self.trace_perimeter(style=StyleCSS("blank"),debug=False)
    #self.builder.add_style(pathstyle)

    pathLayer = ""
    for path in paths:
        if path.points[0].distance(path.points[-1]) < 40:
            cl = path.closed()
        else:
            cl = path
        
        pathLayer += "<path d=\""
        if smooth:
            curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
            smooth = curved.smooth(iterations=1)
            pathLayer += smooth.to_svg_path( close=True)
        else:
            
            pathLayer += cl.to_svg_path( close=True)
        pathLayer += f"\" style=\"fill:{fill}\""
        pathLayer += "/>"
        pathLayer += "\n"
    

    return pathLayer

    
    flowData = np.zeros(len(self.hexGrid.hexes)) - 1

    for i in self.hexes:
        #print(i)
        flowData[i] = 1

    flowData = [int(x) for x in flowData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(self)
    patterns = patternGen.ballDensity(3,fills=fills)  # 5 levels
    self.makeOverlay(flowData, patterns)

In [ ]:
@patch
def fillPatternInverted(self:HexRegion, pattern:SVGBuilder,dest:SVGBuilder, smooth=False):
    """Fill OUTSIDE a region with a pattern."""
    
    # Get the region boundary
    paths, gaps = self.trace_perimeter(style=StyleCSS("blank"), debug=False)
    
    # Create mask definition
    mask_id = f"mask_{id(self)}"
    mask_def = f'<mask id="{mask_id}">'
    mask_def += '<rect x="0" y="0" width="10000" height="10000" fill="white"/>'
    
    # Add black paths for the region (these will be "cut out")
    for path in paths:
        cl = path.closed() if path.points[0].distance(path.points[-1]) < 40 else path
        mask_def += '<path d="'
        if smooth:
            curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
            smooth_path = curved.smooth(iterations=1)
            mask_def += smooth_path.to_svg_path(close=True)
        else:
            mask_def += cl.to_svg_path(close=True)
        mask_def += '" fill="black"/>'
    
    mask_def += '</mask>'
    
    # Add mask to definitions
    self.hexGrid.builder.add_definition_raw(mask_def)
    
    # Create filled rect with mask applied
    patName = pattern.attributes['id']
    fill = f"url(#{patName})"
    
    return f'<rect x="0" y="0" width="10000" height="10000" fill="{fill}" mask="url(#{mask_id})"/>'


In [ ]:
@patch 
def demoFill(self:TerraDemo): 
    """Practice building up coord."""

    sampleMap = self.sanFran()
    peaks = sampleMap.find_peaks(7,0)
    start = peaks[0]
    region = HexRegion(hexes=set([start]),  hex_grid=sampleMap.hexGrid)
    #region.add(start)
    #print(region.perimeter())
    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(8,fills=fills)

    for level in range(3):

        levels = [x for x in range(len(sampleMap.elevations)-1) if sampleMap.elevationLevel(x) == (level-1)]

        region = HexRegion(hexes=set(levels), hex_grid=sampleMap.hexGrid)


        print(f"level {level} count {len(region.hexes)}")
        

        # Show the base terrain
        sampleMap.colorMap()
        sampleMap.hexGrid.update()
        #sampleMap.dot(region.perimeter())

        
          # 5 levels
        myPat = patterns[level+2]
        sampleMap.hexGrid.builder.add_definition(myPat)
        over = region.fillPattern(myPat,True)
        sampleMap.builder.adjust(f"level_{level}",over)
        
    sampleMap.builder.adjust("root","")
    #print(sampleMap.hexGrid.builder.xml())
    #return

    return sampleMap.hexGrid.builder.show()



In [ ]:
#| export
class SVGMask(Generatable):
    """SVG mask definition that can be added to SVGBuilder definitions"""
    
    def __init__(self, mask_id: str, width: int = 10000, height: int = 10000):
        self.mask_id = mask_id
        self.width = width
        self.height = height
        self.paths = []  # List of (path_d, fill_color) tuples
        self.background_color = "white"  # White = show, black = hide
    
    def add_path(self, path_d: str, fill: str = "black", smooth: bool = False):
        """Add a path to the mask. Black areas will be hidden."""
        self.paths.append((path_d, fill))
        return self
    
    def set_background(self, color: str):
        """Set mask background color (white=show, black=hide)"""
        self.background_color = color
        return self
    
    def generate(self) -> str:
        """Generate the mask definition XML"""
        ret = f'<mask id="{self.mask_id}">\n'
        ret += f'  <rect x="0" y="0" width="{self.width}" height="{self.height}" fill="{self.background_color}"/>\n'
        
        for path_d, fill in self.paths:
            ret += f'  <path d="{path_d}" fill="{fill}"/>\n'
        
        ret += '</mask>'
        return ret


In [ ]:
#| export
@patch
def fillPatternInverted(self: HexRegion, pattern: SVGBuilder, smooth: bool = False):
    """Fill OUTSIDE a region with a pattern using a mask."""
    
    # Get the region boundary
    paths = self.trace_perimeter(style=StyleCSS("blank"), debug=False)
    
    # Create mask
    mask_id = f"mask_{id(self)}"
    mask = SVGMask(mask_id)
    
    # Add paths to mask (black = hide these areas)
    for path in paths:
        cl = path.closed() if path.points[0].distance(path.points[-1]) < 40 else path
        
        if smooth:
            curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
            smooth_path = curved.smooth(iterations=1)
            path_d = smooth_path.to_svg_path(close=True)
        else:
            path_d = cl.to_svg_path(close=True)
        
        mask.add_path(path_d, fill="black")
    
    # Add mask to builder definitions
    self.hex_grid.builder.add_definition(mask)
    
    # Create filled rect with mask applied
    patName = pattern.attributes['id']
    fill = f"url(#{patName})"
    
    return f'<rect x="0" y="0" width="10000" height="10000" fill="{fill}" mask="url(#{mask_id})"/>'


In [ ]:
#| export
@patch
def hatchLines(self: TerrainPatterns, angle: float = 45, spacing: float = 8, 
               stroke_width: float = 1.5, color: str = "#3d9fc0ff") -> SVGBuilder:
    """Create diagonal line hatch pattern."""
    
    # Pattern tile needs to be large enough for one full line repeat
    size = spacing * 2
    
    # Calculate line endpoints based on angle
    # For 45°, lines go from bottom-left to top-right
    body = f'''
    <g>
        <line x1="0" y1="{size}" x2="{size}" y2="0" 
              stroke="{color}" stroke-width="{stroke_width}"/>
        <line x1="-{size}" y1="{size}" x2="{size}" y2="-{size}" 
              stroke="{color}" stroke-width="{stroke_width}"/>
    </g>
    '''
    
    aBuilder = SVGBuilder()
    aBuilder.blockTag = "pattern"
    aBuilder.width = size
    aBuilder.height = size
    aBuilder.attributes = {
        'id': 'hatch_lines',
        'patternUnits': 'userSpaceOnUse',
        'patternTransform': f'rotate({angle})'
    }
    aBuilder.updateLayers([body])
    
    return aBuilder


### adding elements

In [ ]:
#| export
@patch
def island(self:TerraDemo):
    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=15,path = "volcano.svg")
    
    sampleMap.elevations += sampleMap.volcano(center=267,adjusted=500,num_rings=6,variability=0.5,initial_threshold=0.4)

    levels = [x for x in range(len(sampleMap.elevations)-1) if sampleMap.elevationLevel(x) >= 0]

    region = HexRegion(hexes=set(levels), hex_grid=sampleMap.hexGrid)
    fills=["#3300ffeb","#3d9fc0ff"]
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(4,fills=fills)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    myPat = patterns[1]
    #myPat = patternGen.hatchLines(angle=25, spacing=8, stroke_width=1.5, color="#3d9fc0ff")

    sampleMap.hexGrid.builder.add_definition(myPat)
    over = region.fillPatternInverted(myPat,True)
    #sampleMap.builder.adjust(f"root","")
    sampleMap.builder.adjust(f"ocean",over)
    return sampleMap

@patch
def demoAquatic(self:TerraDemo):
    sampleMap = self.island()

    

    #print(sampleMap.hexGrid.builder.xml())
    

    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoAquatic()

In [ ]:
SeaPatterns = SVGPatternLoader(patterns_dir="data/patterns/nautical/")
SeaPatterns.get_available_patterns()

['merleft_2.svg',
 'ten_8.svg',
 'compass_1.svg',
 'ten_5.svg',
 'ten_6.svg',
 'ten_4.svg',
 'ten_2.svg',
 'merright_1.svg',
 'ten_1.svg',
 'ten_9.svg',
 'ten_3.svg',
 'compass_2.svg',
 'merleft_1.svg',
 'compass_3.svg',
 'ten_7.svg',
 'merright_2.svg',
 'ten+10.svg']

In [ ]:
#| export
@patch
def mapElement(sampleMap:Terrain,bounds:MapCord,name="compass_1",prefix="merright",style=StyleCSS("base", 
                        fill="#27ae60",  # Green
                        stroke="#c12121ff")): 
    
    # Show the terrain first
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    dim = bounds.dimensons.width
    xoffset = bounds.origin.x
    yoffset = bounds.origin.y

    
    #
    #name = 'compass_1.svg' #'merright_1svg'.
    aPat = SeaPatterns.load_pattern(name + '.svg', f"{name}_aid", prefix=prefix)

    # Override to green - all paths will have class="merright_path"
    green_style = StyleCSS(f"{prefix}_path", 
                        fill=style.properties["fill"],  # Green
                        stroke=style.properties["stroke"])
    aPat.attributes['patternUnits'] = 'userSpaceOnUse'
    aPat.attributes['patternContentUnits'] = 'objectBoundingBox'
    aPat.attributes['patternTransform'] = f'translate({xoffset},{yoffset}) scale({dim/aPat.width})'
    aPat.add_style(green_style)
    #aPat.add_style(green_style, prefix="merright")
    
    sampleMap.hexGrid.builder.add_definition(aPat)
    over = f'<rect x="{xoffset}" y="{yoffset}" width="{dim}" height="{dim}" fill="#0080ff1f"   opacity="0.8"/>'
    over = f'<rect x="{xoffset}" y="{yoffset}" width="{dim}" height="{dim}" fill="url(#{name}_aid)"   opacity="0.8"/>'
    return over
    

    

    return sampleMap.hexGrid.builder.show()

In [ ]:
@patch
def demoAquaticDetails(self:TerraDemo): 
    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap = Terrain(myBounds, radius=15, path="volcano.svg")
    
    sampleMap.elevations += sampleMap.volcano(center=267, adjusted=500, num_rings=6, variability=0.5, initial_threshold=0.4)

    # Show the terrain first
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    dim = 150
    xoffset = 600
    yoffset = 500

    
    # Load and add the compass pattern with smaller scale
    myColors = StyleCSS("base", fill="#ae2778ff",   stroke="#c12121ff")
    
    over = sampleMap.mapElement(MapRect(MapCord(xoffset,yoffset),MapSize(dim,dim)),name="merright_1",prefix="merright",style=myColors)
    
    # Add compass as overlay instead of replacing everything
    #sampleMap.hexGrid.builder.adjust("root","")
   
    sampleMap.hexGrid.builder.adjust("compass2", over)

    myColors = StyleCSS("base", fill="#8cae27ff",   stroke="#c12121ff")
    
    over = sampleMap.mapElement(MapRect(MapCord(20,yoffset),MapSize(160,220)),name="ten_4",prefix="ten_4",style=myColors)
    sampleMap.hexGrid.builder.adjust("ten", over)


    file_name = "tmp/tp_AquaticDetail_demo.svg"
    with open(file_name, 'w') as file_object:
        file_object.write(sampleMap.hexGrid.builder.xml())
    #print(sampleMap.hexGrid.builder.xml())
    

    return sampleMap.hexGrid.builder.show()

In [ ]:
TerraDemo().demoAquaticDetails()

In [ ]:
name = 'merright_1.svg'
aPat = SeaPatterns.load_pattern(name, "compass_1")
print(StyleCSS.generate(SeaPatterns.find_css(name)))

In [ ]:
name = 'merright_1.svg'
aPat = SeaPatterns.load_pattern(name, "compass_1")
print(StyleCSS.generate(SeaPatterns.find_css(name)))